<a href="https://colab.research.google.com/github/ggwnsghgg/book_project/blob/main/%EC%B1%85_%EC%B6%94%EC%B2%9C%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install pymysql
!pip install konlpy

In [9]:
import pymysql
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import konlpy


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# from google.colab import drive
# drive.mount('/content/drive')



#데이터 가져오기
db = pymysql.connect(host='220.86.191.110', port=3306, user='playdata', password='playdata123!', db='bookdata', charset='utf8')

curs = db.cursor(pymysql.cursors.DictCursor)  

query = '''
select *
    from books_data'''
df_books_data = pd.read_sql(query, db)

In [ ]:
# 구글 드라이브 사용시

# path = '/content/drive/MyDrive/data/'
# df_books_data = pd.read_csv(os.path.join(path, 'book_data.csv'), error_bad_lines=False)

In [10]:
df_books_data

,id,title,author,pub,date,price,review,rating,story,img,genre
0,1,요리코를 위해,노리즈키 린타로,모모,2020.03,13500,43,9.2,스스로 악마이자 신이 되어 딸을 살해한 남자를 심판한 아버지.비극의 출발이자 마지막...,http://image.yes24.com/goods/89392792/L,추리/미스터리
1,2,그녀는 다 계획이 있다,히가시노 게이고,하빌리스,2021.02,14220,13,9.4,하나야 보석점 살인사건을 둘러싼 과거와 미래의 사건경쾌하고 리듬감 있는 터치의 히가...,http://image.yes24.com/goods/97561319/L,추리/미스터리
2,3,블랙 쇼맨과 이름 없는 마을의 살인,히가시노 게이고,알에이치코리아(RHK),2020.11,16200,54,9.1,"아버지가 살해당했다, 그것도 결혼 소식을 알린 직후에. 행복하다고 느끼는 순간, 비...",http://image.yes24.com/goods/95712763/L,추리/미스터리
3,4,돌이킬 수 없는 약속,야쿠마루 가쿠,북플라자,2017.02,13500,217,8.2,"제51회 에드가와란포상 수상 작가 야쿠마루 가쿠의 미스터리 추리 소설. ""내 딸을 ...",http://image.yes24.com/goods/35552641/L,추리/미스터리
4,5,루팡의 딸,요코제키 다이,북플라자,2019.09,13500,31,8.5,도둑 집안의 대부가 변사체로 발견되었다!도둑 집안의 대부 이와오에게는 하나코라는 손...,http://image.yes24.com/goods/78860364/L,추리/미스터리
...,...,...,...,...,...,...,...,...,...,...,...
79770,79771,하루만 사랑해 2,,파피루스(디앤씨미디어),2003.06,8100,0,0.0,어느 날 서원은 횡단보도에서 교통사고를 당한다. 상대는 인천상고 짱인 노란머 민하루...,http://image.yes24.com/goods/375484/L,로맨스
79771,79772,서울 여자와 대구 남자가 만날 때,정민,영상노트,2003.06,8100,0,0.0,,http://image.yes24.com/goods/376458/L,로맨스
79772,79773,kiss me...,,현무,2003.06,7200,2,5.0,,http://image.yes24.com/goods/377351/L,로맨스
79773,79774,어린 엄마 2,임은희,반디출판사,2003.07,8100,0,0.0,,http://image.yes24.com/goods/377607/L,로맨스


In [126]:

# 장르와 타이틀 값을 이용한 books_df 제작

df = pd.DataFrame(df_books_data['id'], columns=['id'])
df1 = pd.DataFrame(df_books_data['story'], columns=['story'])
df2 = pd.DataFrame(df_books_data['genre'], columns=['genre'])
books_df = pd.concat([df,df1,df2],axis=1)

# books_df = bookdf.set_index('id')
books_df

,id,story,genre
0,1,스스로 악마 이자 신 딸 살해 남자 심판 아버지 비극 출발 이자 마지막 끝내 실체 ...,추리/미스터리
1,2,하나 보석 점 살인 사건 과거 미래 사건 경쾌하다 리듬 감 있다 터치 게이고 복고 ...,추리/미스터리
2,3,아버지 살해 당하다 그것 결혼 소식 직후 행복하다 순간 비극 시작 대기업 취직 후 ...,추리/미스터리
3,4,제 에드가 란 포상 수상 작가 야쿠 마루 가쿠 미스터리 추리 소설 내 딸 살해 놈 ...,추리/미스터리
4,5,도둑 집안 대부 변 사체 발견 도둑 집안 대부 이 오 하나 코 손녀 딸 있다 하나 ...,추리/미스터리
...,...,...,...
79770,79771,날 서원 횡단보도 교통사고 당하다 상대 인천 상고 짱 노랗다 머 민 하루 놈 오히려...,로맨스
79771,79772,,로맨스
79772,79773,,로맨스
79773,79774,,로맨스


In [127]:
#스토리의 키워드를 뽑기위해 명사와 형용사만 가져오자
story_words = books_df['story'].to_numpy()
for i, document in enumerate(story_words):
    okt = konlpy.tag.Okt()
    clean_words = []
    for word in okt.pos(document, stem = True): #어간 뽑자
        if word[1] in ['Noun']: #명사만 가져오기
            clean_words.append(word[0])
    #print(clean_words)
    document = ' '.join(clean_words)
    #print(document)
    story_words[i] = document

In [128]:
#넘파이 배열 데이터프레임 화
df_story_words=pd.DataFrame(story_words,
            columns = ['story_words'])

In [129]:
#혹시모를 중복 제거하자
books_df = books_df.drop_duplicates(['story'])

In [130]:
#결측치 제거
books_df = books_df.dropna(axis=0)

In [131]:
books_df

,id,story,genre
0,1,스스로 악마 이자 신 딸 살해 남자 심판 아버지 비극 출발 이자 마지막 끝내 실체 ...,추리/미스터리
1,2,하나 보석 점 살인 사건 과거 미래 사건 리듬 감 터치 게이고 복고 미스터리 교코 ...,추리/미스터리
2,3,아버지 살해 그것 결혼 소식 직후 순간 비극 시작 대기업 취직 후 약혼자 결혼식 준...,추리/미스터리
3,4,제 에드가 란 포상 수상 작가 야쿠 마루 가쿠 미스터리 추리 소설 내 딸 살해 놈 ...,추리/미스터리
4,5,도둑 집안 대부 변 사체 발견 도둑 집안 대부 이 오 하나 코 손녀 딸 하나 코 자...,추리/미스터리
...,...,...,...
79764,79765,일생 단 한번 사랑 아르바이트 노래 클럽 닐 만난 조스 서로 운명 임 그 뭔가 비밀...,로맨스
79765,79766,해리 스포츠 담당 기자 스티브 여자 인기 최고 매너 남 해리 엇 꿈 치어리더 것 때...,로맨스
79766,79767,얼음 공주 전문가 회사 스캔들 위해 가짜 연인 행세 지나 플린트 얼음 공주 지나 취...,로맨스
79767,79768,커플 매니저 민디 순종 아내 때문 수 디 가장 커플 매니저 민디 도전 과제 순종 아...,로맨스


In [132]:
df_books = pd.concat([books_df, df_story_words], axis = 1)

In [133]:
df_books.head()

,id,story,genre,story_words
0,1.0,스스로 악마 이자 신 딸 살해 남자 심판 아버지 비극 출발 이자 마지막 끝내 실체 ...,추리/미스터리,스스로 악마 이자 신 딸 살해 남자 심판 아버지 비극 출발 이자 마지막 끝내 실체 ...
1,2.0,하나 보석 점 살인 사건 과거 미래 사건 리듬 감 터치 게이고 복고 미스터리 교코 ...,추리/미스터리,하나 보석 점 살인 사건 과거 미래 사건 리듬 감 터치 게이고 복고 미스터리 교코 ...
2,3.0,아버지 살해 그것 결혼 소식 직후 순간 비극 시작 대기업 취직 후 약혼자 결혼식 준...,추리/미스터리,아버지 살해 그것 결혼 소식 직후 순간 비극 시작 대기업 취직 후 약혼자 결혼식 준...
3,4.0,제 에드가 란 포상 수상 작가 야쿠 마루 가쿠 미스터리 추리 소설 내 딸 살해 놈 ...,추리/미스터리,제 에드가 란 포상 수상 작가 야쿠 마루 가쿠 미스터리 추리 소설 내 딸 살해 놈 ...
4,5.0,도둑 집안 대부 변 사체 발견 도둑 집안 대부 이 오 하나 코 손녀 딸 하나 코 자...,추리/미스터리,도둑 집안 대부 변 사체 발견 도둑 집안 대부 이 오 하나 코 손녀 딸 하나 코 자...


In [139]:
test_list = list(np.array(df_books['story'].tolist()))

In [142]:
print(test_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [144]:
txt = str(test_list)
print(txt)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [62]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [145]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

s = PorterStemmer()
words=word_tokenize(txt)
print(words)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [146]:
words

['[',
 "'스스로",
 '악마',
 '이자',
 '신',
 '딸',
 '살해',
 '남자',
 '심판',
 '아버지',
 '비극',
 '출발',
 '이자',
 '마지막',
 '끝내',
 '실체',
 '파악',
 '수',
 '딸',
 '진실',
 '탐정',
 '모두',
 '누군가',
 '세팅',
 '무대',
 '위',
 '인형',
 '괴물',
 '누구',
 '그',
 '괴물',
 '누구',
 '세',
 '여학생',
 '리코',
 '공원',
 '변',
 '사체',
 '발견',
 "'",
 ',',
 "'하나",
 '보석',
 '점',
 '살인',
 '사건',
 '과거',
 '미래',
 '사건',
 '리듬',
 '감',
 '터치',
 '게이고',
 '복고',
 '미스터리',
 '교코',
 '부자',
 '계획',
 '위해',
 '파티',
 '출근',
 '컴패',
 '이',
 '날',
 '하나',
 '보석',
 '점',
 '고객',
 '감사',
 '파티',
 '뒤',
 '직장',
 '동료',
 '에리',
 '호텔',
 '밀실',
 '채',
 '발견',
 '교코',
 '자신',
 "'",
 ',',
 "'아버지",
 '살해',
 '그것',
 '결혼',
 '소식',
 '직후',
 '순간',
 '비극',
 '시작',
 '대기업',
 '취직',
 '후',
 '약혼자',
 '결혼식',
 '준비',
 '마',
 '경찰서',
 '온',
 '통',
 '전화',
 '그녀',
 '고향',
 '간다',
 '이제',
 '너',
 '일만',
 '거',
 '던',
 '아버지',
 '전화',
 '생전',
 '마지막',
 '통화',
 '경',
 "'",
 ',',
 "'제",
 '에드가',
 '란',
 '포상',
 '수상',
 '작가',
 '야쿠',
 '마루',
 '가쿠',
 '미스터리',
 '추리',
 '소설',
 '내',
 '딸',
 '살해',
 '놈',
 '후',
 '그',
 '약속',
 '내',
 '전',
 '재산',
 '주인공',
 '과거',
 '죄',
 '전',
 '노파',
 '수

In [150]:
for word in list(set(words)):
    word_cnt[word] = words.count(word)
print(word_cnt)

{('기적', 'Noun'): 1, "'쿠란": 1, '마전': 6, '가재': 2, '마루타': 1, '해풍': 1, "'폭주": 3, '곧잘': 8, '샤안샤': 2, '호진': 4, '우태': 3, '종칠': 5, '콜린': 23, '이스라엘': 8, '메로': 1, '이키': 2, '김순': 1, '가쁘': 16, '밀른': 2, '효심': 3, '덕흥리': 1, '인과': 26, '키치': 2, '신겐': 9, '백천이': 1, '섭정': 7, '베르키': 2, '것임': 18, '꽃': 445, '아벨라': 4, '그다지': 20, '경성': 31, '후속': 12, '경민': 13, '덴마크인': 2, '미필': 1, '출산': 14, '시디': 2, '설거지': 7, '륵': 2, '정적': 27, '덩이': 7, '솔솔': 4, '페르티': 1, "'옥루몽": 1, '라흘란': 1, '박정수': 6, '남녀': 224, '각모': 1, '스페인군': 4, '마자': 6, '변천': 4, '썸녀': 1, '이송주': 2, '충절': 4, '괄': 1, '만마겁멸': 1, '취안': 1, "'소개팅": 3, "'한쪽": 3, '김윤자': 1, "'형식": 1, '준비': 336, '율족': 1, '재스퍼': 2, '하르딘': 1, '쪽발이': 2, '생로': 1, '겉보기': 11, '길드장': 1, '블리자드': 18, '야수': 62, "'세": 54, '고영': 6, "'천기": 5, "'망나니": 1, '마타리': 1, '징벌': 12, '문학관': 2, '아르헨': 1, '광합성': 1, '프로포즈': 3, '데이비슨': 1, '오랑우탄': 2, '추밀원': 3, '그리핀': 7, '문지우': 1, '주서윤': 1, "'이산": 3, '환십몽': 2, '건문제': 3, "'질주": 3, '버쉴': 1, "'신임": 2, '용태': 2, '글썽': 3, '실었으': 1, "'카카오": 7, '앰허스터': 2, '할애': 3, "'리디북스":

In [160]:
sorted(word_cnt.items())
tuples.sort(key=lambda t: t[0])

TypeError: ignored

In [52]:
total_count = len(books_df.index)
total_story = list(set([story for sublist in list(map(lambda x: x.split('|'), books_df['story'])) for story in sublist]))

In [53]:
print(f"전체 도서 수: {total_count}")
print(f"장르: {total_story}")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [57]:
print(len(total_story))

30733


In [58]:
story_count = dict.fromkeys(total_story)

for each_story_list in books_df['story']:
    for story in each_story_list.split('|'):
        if story_count[story] == None:
            story_count[story] = 1
        else:
            story_count[story] = story_count[story]+1

KeyError: ignored

In [ ]:
genre_count = dict.fromkeys(total_genres)

for each_genre_list in books_df['genre']:
    for genre in each_genre_list.split('|'):
        if genre_count[genre] == None:
            genre_count[genre] = 1
        else:
            genre_count[genre] = genre_count[genre]+1

In [ ]:
genre_count

{'SF': 1363,
 '공포/스릴러': 3260,
 '로맨스': 14826,
 '무협': 14527,
 '역사': 4524,
 '추리/미스터리': 4152,
 '판타지': 37123}

In [ ]:
for each_genre in genre_count:
    genre_count[each_genre] = np.log10(total_count/genre_count[each_genre])
  
genre_count

{'SF': 1.7673709570308744,
 '공포/스릴러': 1.3886492127976091,
 '로맨스': 0.730842817079825,
 '무협': 0.7396908763353855,
 '역사': 1.2463442166121304,
 '추리/미스터리': 1.2836094680251466,
 '판타지': 0.3322237475062316}

In [ ]:
story_count = dict.fromkeys(total_genres)

for each_genre_list in books_df['genre']:
    for genre in each_genre_list.split('|'):
        if genre_count[genre] == None:
            genre_count[genre] = 1
        else:
            genre_count[genre] = genre_count[genre]+1

In [ ]:
# create genre representations
genre_representation = pd.DataFrame(columns=sorted(total_genres), index=books_df.index)
for index, each_row in tqdm(books_df.iterrows()):
    dict_temp = {i: genre_count[i] for i in each_row['genre'].split('|')}
    row_to_add = pd.DataFrame(dict_temp, index=[index])
    genre_representation.update(row_to_add)

genre_representation

48720it [09:02, 86.25it/s]

In [47]:
from math import log # IDF 계산을 위해

In [48]:
docs = books_df['genre'] 
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

In [49]:
N = len(docs) # 총 문서의 수

def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df + 1))

def tfidf(t, d):
    return tf(t,d)* idf(t)

In [50]:
result = []
for i in range(N): # 각 문서에 대해서 아래 명령을 수행
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]        
        result[-1].append(tf(t, d))

tf_ = pd.DataFrame(result, columns = vocab)
tf_

KeyError: ignored

In [ ]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index = vocab, columns = ["IDF"])
idf_

In [ ]:
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]

        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

In [ ]:
#장르의 키워드를 뽑기위해 명사와 형용사만 가져오자
story_words = df_books_data['genre'].to_numpy()
for i, document in enumerate(story_words):
    okt = konlpy.tag.Okt()
    clean_words = []
    for word in okt.pos(document, stem = True): #어간 뽑자
        if word[1] in ['Noun', 'Adjective']: #명사, 형용사만 가져와
            clean_words.append(word[0])
    #print(clean_words)
    document = ' '.join(clean_words)
    #print(document)
    story_words[i] = document